Import all the necessary packages:

In [ ]:
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

Use Pandas to read the CSV file into a Pandas dataframe and use the dataframe to access the EEG files

In [ ]:
df = pd.read_csv ('train_info.csv')
if 0:
    print(f"\n# lines = {len(df)}\n")
    print("Columns:\n")
    print(df.columns)
    print("\n")
    print(df.info())
    print(df.describe())

print(df)

Get the metadata for the group of seizure-present files and the group of seizure-absent files from the dataframe:

In [ ]:
szIdx = df.index[df['sz_present'] == 1].tolist()
nonIdx = df.index[df['sz_present'] == 0].tolist()
sz = df.loc[df['sz_present']==1]
nonsz = df.loc[df['sz_present']==0]
print('Seizure-present files:')
print(sz)
print('\nSeizure-absent files:')
print(nonsz)

Read data for seizure-present EEGs from binary files as indicated in the metadata in the Pandas dataframe:

In [ ]:
dt = np.uint16
szData = np.zeros((8, 307200))
idx = 0
for s in szIdx:
    path = join('train_files', df['file_id'][s] + '.dat')
    print(path)
    infile = open(path, 'rb')
    temp = np.array([])
    while True:
        buf = infile.read(1024*1024)
        if buf:
            a = np.frombuffer(buf,dtype=dt)
            temp = np.append(temp, a, axis=0)
            print(f'.', end ='', flush = True)
        else:
            break
    infile.close()
    print(np.shape(temp))
    szData[idx][:] = temp
    idx += 1
print(np.shape(szData))

Read in seizure-absent EEGs from binary data files:

In [ ]:
nonszData = np.zeros((8, 307200))
idx = 0
for n in nonIdx:
    path = join('train_files', df['file_id'][n] + '.dat')
    print(path)
    infile = open(path, 'rb')
    temp = np.array([])
    while True:
        buf = infile.read(1024*1024)
        if buf:
            a = np.frombuffer(buf,dtype=dt)
            temp = np.append(temp, a, axis=0)
            print(f'.', end ='', flush = True)
        else:
            break
    infile.close()
    print(np.shape(temp))
    nonszData[idx][:] = temp
    idx += 1
print(np.shape(nonszData))

Convert the EEGs from quantized unsigned ints to microvolts:

In [ ]:
szData_uv = szData*(175*2)/65535 - 175
nonszData_uv = nonszData*(175*2)/65535 - 175
print(np.max(szData_uv))
print(np.min(szData_uv))
print(np.max(nonszData_uv))
print(np.min(nonszData_uv))

Specify the signal processing parameters:

In [ ]:
fs = 512 # Hz
T = 1/fs # seconds

# time axis
t = np.arange(np.size(szData_uv,1))/fs

Plot full EEGs that contain seizures:

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    x = t
    y = szData_uv[cnt][:]

    t_start = df['t_start'][s]
    t_end = df['t_end'][s]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0, label='EEG')
    ax[cnt].plot(t_start, 0, "o", linewidth=2.0, color="yellow", label='sz start')
    ax[cnt].plot(t_end, 0, "o", linewidth=2.0, color="red", label='sz end')
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    ax[cnt].legend(loc='upper right')
    cnt += 1
fig.tight_layout()
plt.show()

Three components can be observed: the seizures, between the yellow and red dots, the pre-ictal period, and the post-ictal period. The activity in the post-ictal period is generally greater than the activity during the pre-ictal period. Note that seizures have different durations. Five of the recordings (train sets 01, 03, 10. 12 and 14) have amplitude clipping due to dynamic range saturation. The clipping precludes waveform analysis with those recordings. 

Extract the actual seizure activity each of the full recordings and plot them

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
t_start_sample = [None] * np.size(szData_uv,0)
t_stop_sample = [None] * np.size(szData_uv,0)
cnt = 0
for s in szIdx:
    t_start = df['t_start'][s]
    t_end = df['t_end'][s]
    t_start_sample[cnt] = int(np.floor(t_start*fs))
    t_stop_sample[cnt] = int(np.ceil(t_end*fs))
    x = t[t_start_sample[cnt]:t_stop_sample[cnt]]
    y = szData_uv[cnt][t_start_sample[cnt]:t_stop_sample[cnt]]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()


Plot the full EEGs that do not contain seizures:

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(nonszData_uv,0),1,figsize=(8, 15))
cnt = 0
for n in nonIdx:
    x = t
    y = nonszData_uv[cnt][:]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG ' + df['file_id'][n] + ' - Seizure absent')  
    cnt += 1
fig.tight_layout()
plt.show()

To identify waveform features, plot the data at a higher time resolution by plotting short segments. Plot the first second (512 samples) of the seizure-absent EEGs as a comparison to the seizure waveforms: 

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(nonszData_uv,0),1,figsize=(8, 15))
cnt = 0
for n in nonIdx:
    x = t[t_start_sample[cnt]: (t_start_sample[cnt] + fs*1)]
    y = nonszData_uv[cnt][t_start_sample[cnt]:(t_start_sample[cnt] + fs*1)]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG, 1-Second Segment - ' + df['file_id'][n] + ' - Seizure absent')  
    cnt += 1
fig.tight_layout()
plt.show()

The question is how the seizure EEG recordings compare to these apparantly "normal" EEG recordings. The train_11 and train_13 EEG recordings look too regular to be biological phenomena. One suspects that they may be recordings of electronics, like the lights or a component with the EEG sensor itself. In fact, it is mostly likely the lights at 60 Hz. The PSDs of the seizure-absent data have spikes at 60 Hz. One could use a notch filter to remove it.

Plot the first second (512 samples) of the seizures to see the seizure recordings in higher time resolution to try to see a waveform: 

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    x = t[t_start_sample[cnt]:t_start_sample[cnt] + fs*1]
    y = szData_uv[cnt][t_start_sample[cnt]:t_start_sample[cnt] + fs*1]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG, Beginning of seizure - ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()

The Generalized Absence seizure (Train_15) has the most characteristic pattern that jumps out right away. There are periodic spikes at a certain frequency that can be characterized. 

Extend the plot to the first 2 seconds (1024 samples) of the seizures to see the seizure recordings in higher time resolution to try to see a waveform: 

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    x = t[t_start_sample[cnt]:t_start_sample[cnt] + fs*2]
    y = szData_uv[cnt][t_start_sample[cnt]:t_start_sample[cnt] + fs*2]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG, Beginning of seizure - ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()

The Generalized Absence seizure (Train_15) has the most characteristic pattern that jumps out right away. There are periodic spikes (even double spikes) that occur at a certain frequency that can be characterized. 

Plot 1 second during the middle of each seizure:

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    duration = df['t_end'][s]- df['t_start'][s]
    x = t[t_start_sample[cnt] + int((duration/2)*fs) :t_start_sample[cnt] + int((duration/2)*fs) + fs*1]
    y = szData_uv[cnt][t_start_sample[cnt] + int((duration/2)*fs) :t_start_sample[cnt] + int((duration/2)*fs) + fs*1]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG - Middle of Seizure - ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()

Plot 1 second just before the end of each seizure:

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    x = t[t_stop_sample[cnt] - fs*2 : t_stop_sample[cnt] - fs*1]
    y = szData_uv[cnt][t_stop_sample[cnt] - fs*2 : t_stop_sample[cnt] - fs*1]

    # plot
    ax[cnt].plot(x, y, linewidth=2.0)
    ax[cnt].grid(True)
    ax[cnt].set_xlabel('Time (s)')
    ax[cnt].set_ylabel('microvolts')
    ax[cnt].set_title('EEG, End of seizure - ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()

Compare train_03 and train_04. They are from the same electrode but one has a seizure and one doesn't. These two examples allow direct comparison between seizure and non-seizure waveforms. (Being from different patients is an uncontrolled factor.)

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(2,1,figsize=(8, 4))
x = t[t_start_sample[2]:t_start_sample[2] + fs*2]
y_03 = szData_uv[2][t_start_sample[2] : t_start_sample[2] + fs*2]
y_04 = nonszData_uv[2][t_start_sample[2] : t_start_sample[2] + fs*2]

# plot
ax[0].plot(x, y_03, linewidth=2.0, label='RE, sz')
ax[0].grid(True)
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('microvolts')
ax[0].set_title('EEG RF - seizure - ' + df['file_id'][2] + ' - ' + df['sz_type'][2])  

ax[1].plot(x, y_04, linewidth=2.0, label='RE, non-sz')
ax[1].grid(True)
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('microvolts')
ax[1].set_title('EEG RF - nonseizure - ' + df['file_id'][3])  
fig.tight_layout()
plt.show()

Train_03 and train_04 in the plot above are from the same electrode but one has a seizure and one doesn't. These two examples allow direct comparison between seizure and non-seizure waveforms. (Being from different patients is an uncontrolled factor.) Comparing the two waveforms from the same electrode, the top is during a seizure, the bottom is not during a seizure: The nonseizure waveform has fluctuations with consistent amplitudes. The seizure waveform has periodic spikes of at least two different frequencies. Both EEGs have a lower frequency drift.  

It may be necessary to normalize or standardize recordings from difference hospitals or different individuals. Normalize the EEGs by the estimated noise power.

In [ ]:
# Compute the noise power, entire 10 minute recording
nonsz_var = np.zeros((np.size(nonszData_uv,0)), dtype=float)
cnt = 0
for n in nonIdx:
    nonsz_var[cnt] = np.var(nonszData_uv[cnt][:])
    cnt += 1

# Normalize by the noise power, entire 10 minute recording
nonszData_uv_norm = np.zeros(np.shape(nonszData_uv), dtype=float)
cnt = 0
for n in nonIdx:
    nonszData_uv_norm[cnt][:] = nonszData_uv[cnt][:] / np.sqrt(nonsz_var[cnt])
    # print(np.var(nonszData_uv_norm[cnt][:]))
    cnt += 1
# print(np.var(nonszData_uv_norm[][:]))

In [ ]:
cnt = 0
nonszData_uv_norm_ave_PSD = np.zeros(np.size(nonszData_uv_norm,1))
for s in nonIdx:
    y = nonszData_uv_norm[cnt][:]
    N_abs = np.size(nonszData_uv_norm,1)
    f = fft(y)
    xf_abs = fftfreq(N_abs, T)
    nonszData_uv_norm_ave_PSD = nonszData_uv_norm_ave_PSD +  20*np.log10(abs(f/N_abs))
    cnt += 1

nonszData_uv_norm_ave_PSD = nonszData_uv_norm_ave_PSD/np.size(nonszData_uv_norm,0)

Plot the average PSD for normalized nonseizure EEGs

In [ ]:
plt.style.use('_mpl-gallery')
fig = plt.figure(figsize=(8, 4))
# plot
plt.plot(xf_abs[:N_abs//2], nonszData_uv_norm_ave_PSD[:N_abs//2], linewidth=2.0, label='nonseizure')
plt.grid(True)
plt.ylim((-100, 0)) 
plt.xlabel('Frequency (Hz)')
plt.ylabel('SNR ' r'$\mu v^{2}/Hz$' + ' (dB)')
plt.title('Average PSD of EEGs, Normalized ' + ' - Seizure absent')  

Note the spike at 60 Hz, which is mostly likely from electrical components. One could use a notch filter to remove it.

To normalize seizure data, use samples from the pre-ictal period:

In [ ]:
# Estimate the noise power. Use pre-ictal samples  
sz_var = np.zeros((np.size(szData_uv,0)), dtype=float)
cnt = 0
for s in szIdx:
    # Normalization with pre-ictal samples
    #N = t_stop_sample[cnt] - t_start_sample[cnt] + 1
    #if int(t_start_sample[cnt] - N) < 0:
    #    nse_start_sample = 0
    #else:
    #    nse_start_sample = int(t_start_sample[cnt] - N)
    #nse = szData_uv[cnt][nse_start_sample : t_start_sample[cnt]] # Not inclusive of t_start_sample[cnt]

    # Normalization to unit variance of the seizure component itself
    nse = szData_uv[cnt][t_start_sample[cnt] : t_stop_sample[cnt]] 
    sz_var[cnt] = np.var(nse)
    cnt += 1

# Normalize by the noise power
szData_uv_norm = np.zeros(np.shape(szData_uv), dtype=float)
cnt = 0
for s in szIdx:
    szData_uv_norm[cnt][:] = szData_uv[cnt][:] / np.sqrt(sz_var[cnt])
    # print(np.var(szData_uv_norm[cnt][t_start_sample[cnt] : t_stop_sample[cnt]]))
    cnt += 1

Compare the PSDs of seizure and non-seizure EEGs. Plot PSDs of normalized seizure EEGs over the average PSD of non-seizure EEG. Look for differences between the PSDs of seizure versus non-seizures.

In [ ]:
plt.style.use('_mpl-gallery')
fig, ax = plt.subplots(np.size(szData_uv_norm,0),1,figsize=(8, 15))
cnt = 0
for s in szIdx:
    y = szData_uv_norm[cnt][t_start_sample[cnt]:t_stop_sample[cnt]]

    N = t_stop_sample[cnt] - t_start_sample[cnt] + 1
    f = fft(y)
    xf = fftfreq(N, T)
    # plot
    ax[cnt].plot(xf[:N//2], 20*np.log10(abs(f[:N//2])/N), color = 'steelblue',linewidth=2.0, label='Sz')
    ax[cnt].plot(xf_abs[:N_abs//2], nonszData_uv_norm_ave_PSD[:N_abs//2], color = 'orange', linewidth=2.0, label='Abs')
    ax[cnt].set_ylim((-100, 0)) 
    ax[cnt].grid(True)
    ax[cnt].legend(loc='upper right')
    ax[cnt].set_xlabel('Frequency (Hz)')
    ax[cnt].set_ylabel('SNR ' + r'$\mu v^{2}/Hz$' + ' (dB)')
    ax[cnt].set_title('PSD of EEG, Normalized ' + df['file_id'][s] + ' - ' + df['sz_type'][s])  
    cnt += 1
fig.tight_layout()
plt.show()